
## Introduction

We present an algorithm to measure the Signal Quality of E4 wristband data taken from second year medical residents, deeming the following three measures as nontrivial: spectral entropy, skewness, and perfusion. Spectral entropy is a measure of the Shannon entropy of the probability distribution corresponding to the spectral power distribution normalized. Skewness follows the classic statistical formula for measuring the skewness of a distribution. Perfusion is the difference between the maximum and minimum values of a signal, normalized by the average of the segment. 

In [123]:
import scipy.signal as scisig
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

bvp =  r"C:\Users\Vansh\Downloads\BVP_test.csv" 
eda =  r"C:\Users\Vansh\Downloads\EDA.csv" 

fs_dict = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'label': 700, 'Resp': 700, 'ECG': 700, 'chest': 700} # Frequency dictionary
filepath = bvp # change
data = pd.read_csv(filepath)
time = data.iloc[:, 3]
data = data.iloc[:, 2]
data = data.to_numpy()
#data = data * 1000000

### Spectral Entropy

Given a segment of the signal, we compute the spectral power as a probability density function and determine its Shannon entropy. This is done for each segment, handled in the evaluation function.

In [124]:
def spectral_entropy(segment, signal_name):
   
    f, Pxx_den = scisig.welch(segment, fs_dict[signal_name]) # Welch's Power spectral density.
    #f_store.append(f)
    #Pxx_store.append(Pxx_den)
    scaled_Pxx = Pxx_den/np.sum(Pxx_den)
    N_bins = np.sum((f>1)*(f<3))
    value = scaled_Pxx*np.log(scaled_Pxx)/N_bins
    return (-np.sum(value))
    

### Perfusion
Max subtracted from min of a signal segment divided by the signal segment's average. We pass in a one segment at a time in the evaluation function.

In [126]:
def perfusion(segment, signal_name): 
    avg = segment.mean()
    bb, aa = scisig.butter(8, 8*2/fs_dict[signal_name])      # filter coefs of 8th order Butterworth filter
    y = scisig.filtfilt(bb, aa, segment)                # applying filter forward and backward to segment of signal
    return (100*(np.max(y)-np.min(y))/np.abs(avg))

### Evaluation
Below returns a list for each signal quality measure, where values correspond to measurements taken from a segment of the signal given a time frame (window size). The time series for each measurement is plotted.

In [127]:
def evaluate_quality(x, signal_name, start_time, sample_freq, window_size=4):
    '''
    evaluate quality
    Parameters
    ----------
    x : array
        signal data to evaluate
    signal: string
        signal to evaluate, key from frequency dictionary
    window_size : int
        window size in seconds
        
    Returns
    -------
    Ssqi : list
           skewness measure of segment
    Psqi : list
           perfusion of signal
    SEsqi: list
           spectral entropy of segment
    '''
    Psqi = []
    Ssqi = []
    SEsqi = []
    i_N = window_size*sample_freq;                          # index per window
    f1 = 1; f2=3;                                           # frequency band
    #f_store = []
    #Pxx_store = []
    time_increments = []
    for i in range(len(x)//(i_N)):                          # iterates on each window
        i_s = i*i_N                                         # counter
        i_f = i_s + i_N
        segment = x[i_s:i_f]
        avg = segment.mean()
        stdv = segment.std()
        Ssqi.append(np.sum((segment -avg/stdv)**3)/i_N)
        p = perfusion(segment, signal_name)
        Psqi.append(p)
        se = spectral_entropy(segment, signal_name)
        SEsqi.append(se)
        time_increments.append(time[i_s])                   # every {window_size} seconds
        del avg; del stdv
    
    plot_time_series_SE(SEsqi, time_increments, signal_name, x, window_size, start_time)
    plot_time_series_P(Psqi, time_increments, signal_name, x, window_size, start_time)
    plot_time_series_skewness(Ssqi, time_increments, signal_name, x, window_size, start_time)
    return Ssqi, Psqi, SEsqi

In [128]:
def plot_series_helper(measure, time_increments, signal_name, signal_data, window_size, start_time, measure_name, range, c, limit):
    df_measure = pd.DataFrame(measure, columns=[measure_name])
    df_measure['Seconds'] = time_increments

    
    df_measure[signal_name] = (signal_data[::(window_size*64)])[:-1]
    start_time = start_time // 4
    df_measure = df_measure.iloc[start_time:]
    fig, ax = plt.subplots(figsize=(30,10)) 
    graph1 = df_measure.plot(x = 'Seconds', y = signal_name, ax = ax, color = "black", fontsize=20) 
   
    #signal measures exceeding 1 std from mean have been ommitted
    graph1.set_ylim(signal_data.mean() - signal_data.std(), signal_data.mean() + signal_data.std())

    graph2 = df_measure.plot(x = 'Seconds', y = measure_name, ax = ax, secondary_y = True, color = c, fontsize=20) 
    measure = np.array(measure)
    if limit: 
        graph2.set_ylim(range[0], range[1])
    
    graph1.set_xlabel('Seconds', fontsize = 20)
    graph1.set_ylabel(signal_name, fontsize = 20)
    graph2.set_ylabel(measure_name, fontsize = 20)
    plt.show()

In [129]:
def plot_time_series_SE(measure, time_increments, signal_name, signal_data, window_size, start_time):
    plot_series_helper(measure, time_increments, signal_name, signal_data, window_size, start_time, "SE", [0, 0], "blue", False)


In [130]:
def plot_time_series_P(measure, time_increments, signal_name, signal_data, window_size, start_time):
    plot_series_helper(measure, time_increments, signal_name, signal_data, window_size, start_time, "Perfusion", [0, 10000000], "red", True)

In [131]:
def plot_time_series_skewness(measure, time_increments, signal_name, signal_data, window_size, start_time):
    plot_series_helper(measure, time_increments, signal_name, signal_data, window_size, start_time, "Skewness", [-5, 5], "orange", True)

### Validation

Given a time frame, we determine the segment's validity using various thresholds relating to the above measurements taken from this segment.

In [132]:
def validate_segment(data, start, end, signal_name, window_size, sample_freq):
    '''
    evaluate quality
    Parameters
    ----------
    start : int
        start time
    end : int
        end time
    signal_name : string
        signal name from frequency dictionary
    window_size : int
        window size in seconds
    sample_freq : int
        sampling frequency of signal
        
    Returns
    -------
    array with 3 values corresponding to skew, se, and perfusion
    1 if needs attention, 0 otherwise
    '''
    s = (start * window_size * sample_freq)
    e = (end * window_size * sample_freq)
    segment = data[s:e]
    bb, aa = scisig.butter(8, 8*2/fs_dict[signal_name])      # filter coefs of 8th order Butterworth filter
    y = scisig.filtfilt(bb, aa, segment)                     # applying filter forward and backward to segment of signal
    f, Pxx_den = scisig.welch(segment, fs_dict[signal_name]) # Welch's Power spectral density.
    
    scaled_Pxx = Pxx_den/np.sum(Pxx_den)
    N_bins = np.sum((f>1)*(f<3))
    value = scaled_Pxx*np.log(scaled_Pxx)/N_bins
    se = -np.sum(value)
    avg = segment.mean()
    stdv = segment.std()
    perfusion = 100*(np.max(y)-np.min(y))/np.abs(avg)
    skewness = np.sum((segment -avg/stdv)**3)/(end-start)
    
    res = {"skewness" : 0, "se" : 0, "perfusion" : 0}
    if abs(skewness) > 1: #placeholders, look into value
        res["skewness"] = 1
    if abs(se) > 0.5:
        res["se"] = 1
    if perfusion > 500000:       #always positive
        res["perfusion"] = 1
    
    return res
    

In [134]:
#passing in BVP signal data to function
signal_type = "BVP"
Ssqi, Psqi, SEsqi = evaluate_quality(data, signal_type, window_size = 4, start_time = 120, sample_freq = fs_dict[signal_type])

In [31]:
# time period 2000 - 8000 seems interesting, lets check
res = validate_segment(data, 2000, 8000, "BVP", 4, 64)
res


{'skewness': 1, 'se': 0, 'perfusion': 1}

### Example Interpretation
Looking at the graphs, we see skewness and perfusion do have incredibly high values in this range, prompting further investigation.